In [1]:
def read_files(filepath):
    with open(filepath) as f:
        str_text=f.read()
        
    return str_text
    

In [2]:
#read_files('../UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt')

In [3]:
#read_files('../UPDATED_NLP_COURSE/06-Deep-Learning/melville-moby_dick.txt')

In [4]:
import spacy

In [5]:
nlp =spacy.load('en_core_web_lg',disable=['parse','tagger','ner'])
nlp.max_length=1198623

In [6]:
def seperate_punc(doc_text):
    return[token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [7]:
d=read_files('../UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt')

In [8]:
tokens=seperate_punc(d)

In [9]:
len(tokens)

11394

In [10]:
train_len=25+1
text_seq=[]
 
for i in range(train_len,len(tokens)):
    seq=tokens[i-train_len:i]
    text_seq.append(seq)

In [11]:
' '.join(text_seq[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [12]:
' '.join(text_seq[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_seq)

In [14]:
seq=tokenizer.texts_to_sequences(text_seq)

In [15]:
tokenizer.index_word

{1: 'the',
 2: 'a',
 3: 'and',
 4: 'of',
 5: 'i',
 6: 'to',
 7: 'in',
 8: 'it',
 9: 'that',
 10: 'he',
 11: 'his',
 12: 'was',
 13: 'but',
 14: 'me',
 15: 'with',
 16: 'as',
 17: 'you',
 18: 'this',
 19: 'at',
 20: 'is',
 21: 'all',
 22: 'for',
 23: 'my',
 24: 'on',
 25: 'be',
 26: "'s",
 27: 'not',
 28: 'from',
 29: 'there',
 30: 'one',
 31: 'up',
 32: 'what',
 33: 'him',
 34: 'so',
 35: 'bed',
 36: 'now',
 37: 'no',
 38: 'about',
 39: 'into',
 40: 'by',
 41: 'were',
 42: 'out',
 43: 'or',
 44: 'harpooneer',
 45: 'had',
 46: 'then',
 47: 'have',
 48: 'an',
 49: 'upon',
 50: 'little',
 51: 'some',
 52: 'old',
 53: 'like',
 54: 'if',
 55: 'they',
 56: 'would',
 57: 'do',
 58: 'over',
 59: 'landlord',
 60: 'thought',
 61: 'room',
 62: 'when',
 63: 'could',
 64: 'here',
 65: 'head',
 66: "n't",
 67: 'night',
 68: 'such',
 69: 'which',
 70: 'man',
 71: 'did',
 72: 'sea',
 73: 'though',
 74: 'time',
 75: 'other',
 76: 'very',
 77: 'go',
 78: 'these',
 79: 'more',
 80: 'first',
 81: 'sort',


In [16]:
tokenizer.word_counts

OrderedDict([('call', 27),
             ('me', 2471),
             ('ishmael', 133),
             ('some', 758),
             ('years', 135),
             ('ago', 84),
             ('never', 449),
             ('mind', 164),
             ('how', 321),
             ('long', 374),
             ('precisely', 37),
             ('having', 142),
             ('little', 767),
             ('or', 950),
             ('no', 1029),
             ('money', 120),
             ('in', 5647),
             ('my', 1812),
             ('purse', 71),
             ('and', 9646),
             ('nothing', 281),
             ('particular', 152),
             ('to', 6497),
             ('interest', 24),
             ('on', 1716),
             ('shore', 26),
             ('i', 7176),
             ('thought', 676),
             ('would', 702),
             ('sail', 104),
             ('about', 1014),
             ('a', 10377),
             ('see', 442),
             ('the', 15566),
             ('watery', 26),
  

In [17]:
vocabulary= len(tokenizer.word_counts)

In [18]:
vocabulary

2709

In [19]:
import numpy as np
seq=np.array(seq)

In [20]:
seq

array([[ 964,   14,  265, ..., 2704,   14,   24],
       [  14,  265,   51, ...,   14,   24,  965],
       [ 265,   51,  263, ...,   24,  965,    5],
       ...,
       [ 960,   12,  168, ...,  264,   53,    2],
       [  12,  168, 2703, ...,   53,    2, 2709],
       [ 168, 2703,    3, ...,    2, 2709,   26]])

In [21]:
from tensorflow.keras.utils import to_categorical

In [22]:
x=seq[:,:-1]

In [23]:
y=seq[:,-1]

In [24]:
y=to_categorical(y,num_classes=vocabulary+1)

In [25]:
seq_len=x.shape[1]

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding

In [31]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model
              

In [32]:
model=create_model(vocabulary+1,seq_len)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 25)            67750     
                                                                 
 lstm_2 (LSTM)               (None, 25, 150)           105600    
                                                                 
 lstm_3 (LSTM)               (None, 150)               180600    
                                                                 
 dense_2 (Dense)             (None, 150)               22650     
                                                                 
 dense_3 (Dense)             (None, 2710)              409210    
                                                                 
Total params: 785,810
Trainable params: 785,810
Non-trainable params: 0
_________________________________________________________________


In [33]:
from pickle import dump,load


In [34]:
model.fit(x,y,batch_size=128,epochs=100,verbose=1)

Epoch 1/100
89/89 [==============================] - 10s 84ms/step - loss: 6.8425 - accuracy: 0.0440
Epoch 2/100
89/89 [==============================] - 8s 86ms/step - loss: 6.3870 - accuracy: 0.0527
Epoch 3/100
89/89 [==============================] - 8s 88ms/step - loss: 6.3454 - accuracy: 0.0527
Epoch 4/100
89/89 [==============================] - 9s 96ms/step - loss: 6.2086 - accuracy: 0.0529
Epoch 5/100
89/89 [==============================] - 9s 102ms/step - loss: 6.1153 - accuracy: 0.0529
Epoch 6/100
89/89 [==============================] - 8s 89ms/step - loss: 5.9914 - accuracy: 0.0594
Epoch 7/100
89/89 [==============================] - 8s 87ms/step - loss: 5.8785 - accuracy: 0.0673
Epoch 8/100
89/89 [==============================] - 8s 88ms/step - loss: 5.7972 - accuracy: 0.0691
Epoch 9/100
89/89 [==============================] - 8s 88ms/step - loss: 5.7208 - accuracy: 0.0699
Epoch 10/100
89/89 [==============================] - 8s 92ms/step - loss: 5.6441 - accuracy: 0.07

89/89 [==============================] - 8s 89ms/step - loss: 3.1549 - accuracy: 0.2535
Epoch 83/100
89/89 [==============================] - 8s 92ms/step - loss: 3.1352 - accuracy: 0.2556
Epoch 84/100
89/89 [==============================] - 8s 89ms/step - loss: 3.1102 - accuracy: 0.2620
Epoch 85/100
89/89 [==============================] - 8s 91ms/step - loss: 3.0979 - accuracy: 0.2641
Epoch 86/100
89/89 [==============================] - 9s 96ms/step - loss: 3.0663 - accuracy: 0.2688
Epoch 87/100
89/89 [==============================] - 8s 94ms/step - loss: 3.0552 - accuracy: 0.2669
Epoch 88/100
89/89 [==============================] - 9s 98ms/step - loss: 3.0267 - accuracy: 0.2737
Epoch 89/100
89/89 [==============================] - 8s 94ms/step - loss: 3.0083 - accuracy: 0.2769
Epoch 90/100
89/89 [==============================] - 8s 93ms/step - loss: 2.9944 - accuracy: 0.2844
Epoch 91/100
89/89 [==============================] - 8s 91ms/step - loss: 2.9632 - accuracy: 0.2891
Epo

In [35]:
model.save('moby4ch.h5')
dump(tokenizer,open('moby4ch','wb'))

In [38]:
model.fit(x,y,epochs=1,verbose=1)

356/356 [==============================] - 12s 35ms/step - loss: 3.5357 - accuracy: 0.1706


In [39]:
model.fit(x,y,batch_size=128,epochs=500,verbose=1)

Epoch 1/500
89/89 [==============================] - 8s 87ms/step - loss: 3.0344 - accuracy: 0.2523
Epoch 2/500
89/89 [==============================] - 8s 88ms/step - loss: 2.7699 - accuracy: 0.3308
Epoch 3/500
89/89 [==============================] - 8s 90ms/step - loss: 2.7038 - accuracy: 0.3593
Epoch 4/500
89/89 [==============================] - 8s 90ms/step - loss: 2.6645 - accuracy: 0.3626
Epoch 5/500
89/89 [==============================] - 8s 92ms/step - loss: 2.6446 - accuracy: 0.3668
Epoch 6/500
89/89 [==============================] - 8s 93ms/step - loss: 2.6298 - accuracy: 0.3695
Epoch 7/500
89/89 [==============================] - 8s 90ms/step - loss: 2.6139 - accuracy: 0.3748
Epoch 8/500
89/89 [==============================] - 8s 90ms/step - loss: 2.6077 - accuracy: 0.3737
Epoch 9/500
89/89 [==============================] - 8s 90ms/step - loss: 2.5925 - accuracy: 0.3716
Epoch 10/500
89/89 [==============================] - 8s 91ms/step - loss: 2.5845 - accuracy: 0.3756

89/89 [==============================] - 8s 90ms/step - loss: 1.7367 - accuracy: 0.5523
Epoch 83/500
89/89 [==============================] - 8s 90ms/step - loss: 1.7239 - accuracy: 0.5545
Epoch 84/500
89/89 [==============================] - 8s 91ms/step - loss: 1.7021 - accuracy: 0.5596
Epoch 85/500
89/89 [==============================] - 8s 91ms/step - loss: 1.6866 - accuracy: 0.5652
Epoch 86/500
89/89 [==============================] - 8s 91ms/step - loss: 1.6863 - accuracy: 0.5647
Epoch 87/500
89/89 [==============================] - 8s 90ms/step - loss: 1.6714 - accuracy: 0.5715
Epoch 88/500
89/89 [==============================] - 8s 90ms/step - loss: 1.6541 - accuracy: 0.5712
Epoch 89/500
89/89 [==============================] - 8s 90ms/step - loss: 1.6480 - accuracy: 0.5749
Epoch 90/500
89/89 [==============================] - 8s 91ms/step - loss: 1.6376 - accuracy: 0.5807
Epoch 91/500
89/89 [==============================] - 8s 90ms/step - loss: 1.6230 - accuracy: 0.5791
Epo

89/89 [==============================] - 8s 92ms/step - loss: 0.9204 - accuracy: 0.7607
Epoch 163/500
89/89 [==============================] - 8s 91ms/step - loss: 0.9216 - accuracy: 0.7606
Epoch 164/500
89/89 [==============================] - 8s 92ms/step - loss: 0.9019 - accuracy: 0.7677
Epoch 165/500
89/89 [==============================] - 8s 91ms/step - loss: 0.8926 - accuracy: 0.7701
Epoch 166/500
89/89 [==============================] - 8s 92ms/step - loss: 0.8900 - accuracy: 0.7713
Epoch 167/500
89/89 [==============================] - 8s 92ms/step - loss: 0.8808 - accuracy: 0.7709
Epoch 168/500
89/89 [==============================] - 8s 92ms/step - loss: 0.8621 - accuracy: 0.7791
Epoch 169/500
89/89 [==============================] - 8s 92ms/step - loss: 0.8576 - accuracy: 0.7784
Epoch 170/500
89/89 [==============================] - 8s 92ms/step - loss: 0.8401 - accuracy: 0.7837
Epoch 171/500
89/89 [==============================] - 8s 92ms/step - loss: 0.8222 - accuracy: 0

89/89 [==============================] - 8s 95ms/step - loss: 0.3318 - accuracy: 0.9295
Epoch 243/500
89/89 [==============================] - 8s 95ms/step - loss: 0.3086 - accuracy: 0.9387
Epoch 244/500
89/89 [==============================] - 9s 105ms/step - loss: 0.2882 - accuracy: 0.9466
Epoch 245/500
89/89 [==============================] - 423s 5s/step - loss: 0.2699 - accuracy: 0.9528
Epoch 246/500
89/89 [==============================] - 9s 101ms/step - loss: 0.2486 - accuracy: 0.9574
Epoch 247/500
89/89 [==============================] - 9s 100ms/step - loss: 0.2380 - accuracy: 0.9619
Epoch 248/500
89/89 [==============================] - 10s 113ms/step - loss: 0.2282 - accuracy: 0.9631
Epoch 249/500
89/89 [==============================] - 9s 105ms/step - loss: 0.2483 - accuracy: 0.9588
Epoch 250/500
89/89 [==============================] - 9s 102ms/step - loss: 0.2689 - accuracy: 0.9511
Epoch 251/500
89/89 [==============================] - 9s 100ms/step - loss: 0.2631 - acc

89/89 [==============================] - 9s 98ms/step - loss: 0.0404 - accuracy: 0.9992
Epoch 323/500
89/89 [==============================] - 9s 98ms/step - loss: 0.0385 - accuracy: 0.9995
Epoch 324/500
89/89 [==============================] - 9s 98ms/step - loss: 0.0371 - accuracy: 0.9996
Epoch 325/500
89/89 [==============================] - 9s 98ms/step - loss: 0.0363 - accuracy: 0.9994
Epoch 326/500
89/89 [==============================] - 9s 98ms/step - loss: 0.1955 - accuracy: 0.9531
Epoch 327/500
89/89 [==============================] - 9s 99ms/step - loss: 0.8653 - accuracy: 0.7581
Epoch 328/500
89/89 [==============================] - 9s 99ms/step - loss: 0.6541 - accuracy: 0.8129
Epoch 329/500
89/89 [==============================] - 9s 99ms/step - loss: 0.3410 - accuracy: 0.9079
Epoch 330/500
89/89 [==============================] - 9s 99ms/step - loss: 0.1614 - accuracy: 0.9658
Epoch 331/500
89/89 [==============================] - 9s 98ms/step - loss: 0.0903 - accuracy: 0

89/89 [==============================] - 8s 93ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 403/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 404/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 405/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 406/500
89/89 [==============================] - 8s 94ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 407/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 408/500
89/89 [==============================] - 8s 94ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 409/500
89/89 [==============================] - 8s 94ms/step - loss: 0.2414 - accuracy: 0.9397
Epoch 410/500
89/89 [==============================] - 8s 94ms/step - loss: 1.6862 - accuracy: 0.6200
Epoch 411/500
89/89 [==============================] - 8s 93ms/step - loss: 0.6213 - accuracy: 0

89/89 [==============================] - 8s 94ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 483/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 484/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 485/500
89/89 [==============================] - 8s 94ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 486/500
89/89 [==============================] - 8s 94ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 487/500
89/89 [==============================] - 8s 94ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 488/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0054 - accuracy: 0.9999
Epoch 489/500
89/89 [==============================] - 8s 94ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 490/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 491/500
89/89 [==============================] - 8s 93ms/step - loss: 0.0048 - accuracy: 1

In [40]:
model.save('moby4ch.h5')

In [41]:
from keras.preprocessing.sequence import pad_sequences

In [73]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
   

    output_text = []
    
    input_text = seed_text
    for i in range(num_gen_words):
        
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
       
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        
        pred_word_ind = np.argmax(model.predict(pad_encoded, verbose=0)[0])
        
        
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    
    return ' '.join(output_text)

In [76]:
import random
random.seed(189)
random_pick=random.randint(0,len(text_seq))
random_seed_text = text_seq[random_pick]

In [77]:
random_seed_text


['to',
 'say',
 'a',
 'word',
 'till',
 'spoken',
 'to',
 'holding',
 'a',
 'light',
 'in',
 'one',
 'hand',
 'and',
 'that',
 'identical',
 'new',
 'zealand',
 'head',
 'in',
 'the',
 'other',
 'the',
 'stranger',
 'entered',
 'the']

In [78]:
seed_text=' '.join(random_seed_text)

In [79]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'room and without looking towards the bed placed his candle a good way off from me on the floor in one corner and then began'